## Database

In [16]:
import sqlite3
from typing import List, Tuple, Optional

# Constants
DB_NAME = 'data/synop/synop.db'

# Schemas
WEATHER_SD_SCHEMA = '''
CREATE TABLE IF NOT EXISTS WeatherSD (
    Record_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Station_ID INTEGER,
    Date DATE,
    Max_Daily_Temperature REAL,
    Min_Daily_Temperature REAL,
    Avg_Daily_Temperature REAL,
    Total_Daily_Precipitation REAL,
    Sunshine_Hours REAL,
    Actinometry REAL,
    Wind_Duration_Over_10m_s REAL,
    FOREIGN KEY (Station_ID) REFERENCES Stations(Station_Code)
);
'''

WEATHER_SDT_SCHEMA = '''
CREATE TABLE IF NOT EXISTS WeatherSDT (
    Record_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Station_ID INTEGER,
    Date DATE,
    Avg_Daily_Overall_Cloudiness REAL,
    Avg_Daily_Wind_Speed REAL,
    FOREIGN KEY (Station_ID) REFERENCES Stations(Station_Code)
);
'''

ELECTRICITY_SCHEMA = '''
CREATE TABLE IF NOT EXISTS ElectricityData (
    Datetime TEXT,
    CarbonIntensityDirect REAL,
    CarbonIntensityLCA REAL,
    LowCarbonPercentage REAL,
    RenewablePercentage REAL
);
'''

# Utility functions for database operations

def create_connection(db_name: str = DB_NAME) -> Optional[sqlite3.Connection]:
    """Establish a connection to the SQLite database."""
    try:
        conn = sqlite3.connect(db_name)
        return conn
    except sqlite3.Error as e:
        print(f"Error connecting to database: {e}")
        return None

def execute_query(conn: sqlite3.Connection, query: str) -> None:
    """Execute a single SQL query."""
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()
    except sqlite3.Error as e:
        print(f"Error executing query: {e}")

def create_table(conn: sqlite3.Connection, schema: str) -> None:
    """Create a table using the provided SQL schema if it doesn't exist."""
    execute_query(conn, schema)

def create_all_tables(conn: sqlite3.Connection) -> None:
    """Create all required tables."""
    create_table(conn, WEATHER_SD_SCHEMA)
    create_table(conn, WEATHER_SDT_SCHEMA)
    create_table(conn, ELECTRICITY_SCHEMA)

# Initialize database and create tables
if __name__ == '__main__':
    conn = create_connection()
    if conn:
        create_all_tables(conn)
        conn.close()

## Data insertion

### imports

In [17]:
import pandas as pd
import sqlite3
from typing import List

### constants

In [18]:
# Constants
DB_NAME = 'data/synop/synop.db'
SD_COLUMNS = ["Kod stacji", "Nazwa stacji", "Rok", "Miesiąc", "Dzień", "Maksymalna temperatura dobowa [°C]", "Status pomiaru TMAX", "Minimalna temperatura dobowa [°C]", "Status pomiaru TMIN", "Średnia temperatura dobowa [°C]", "Status pomiaru STD", "Temperatura minimalna przy gruncie [°C]", "Status pomiaru TMNG", "Suma dobowa opadu [mm]", "Status pomiaru SMDB", "Rodzaj opadu [S/W/ ]", "Wysokość pokrywy śnieżnej [cm]", "Status pomiaru PKSN", "Równoważnik wodny śniegu [mm/cm]", "Status pomiaru RWSN", "Usłonecznienie [godziny]", "Status pomiaru USL", "Czas trwania opadu deszczu [godziny]", "Status pomiaru DESZ", "Czas trwania opadu śniegu [godziny]", "Status pomiaru SNEG", "Czas trwania opadu deszczu ze śniegiem [godziny]", "Status pomiaru DISN", "Czas trwania gradu [godziny]", "Status pomiaru GRAD", "Czas trwania mgły [godziny]", "Status pomiaru MGLA", "Czas trwania zamglenia [godziny]", "Status pomiaru ZMGL", "Czas trwania sadzi [godziny]", "Status pomiaru SADZ", "Czas trwania gołoledzi [godziny]", "Status pomiaru GOLO", "Czas trwania zamieci śnieżnej niskiej [godziny]", "Status pomiaru ZMNI", "Czas trwania zamieci śnieżnej wysokiej [godziny]", "Status pomiaru ZMWS", "Czas trwania zmętnienia [godziny]", "Status pomiaru ZMET", "Czas trwania wiatru >=10m/s [godziny]", "Status pomiaru FF10", "Czas trwania wiatru >15m/s [godziny]", "Status pomiaru FF15", "Czas trwania burzy [godziny]", "Status pomiaru BRZA", "Czas trwania rosy [godziny]", "Status pomiaru ROSA", "Czas trwania szronu [godziny]", "Status pomiaru SZRO", "Wystąpienie pokrywy śnieżnej [0/1]", "Status pomiaru DZPS", "Wystąpienie błyskawicy [0/1]", "Status pomiaru DZBL", "Stan gruntu [Z/R]", "Izoterma dolna [cm]", "Status pomiaru IZD", "Izoterma górna [cm]", "Status pomiaru IZG", "Aktynometria [J/cm2]", "Status pomiaru AKTN"]

SDT_COLUMNS = [
    "Kod stacji", "Nazwa stacji", "Rok", "Miesiąc", "Dzień",
    "Średnie dobowe zachmurzenie ogólne [oktanty]", "Status pomiaru NOS",
    "Średnia dobowa prędkość wiatru [m/s]", "Status pomiaru FWS",
    "Średnia dobowa temperatura [°C]", "Status pomiaru TEMP",
    "Średnia dobowe ciśnienie pary wodnej [hPa]", "Status pomiaru CPW",
    "Średnia dobowa wilgotność względna [%]", "Status pomiaru WLGS",
    "Średnia dobowe ciśnienie na poziomie stacji [hPa]", "Status pomiaru PPPS",
    "Średnie dobowe ciśnienie na pozimie morza [hPa]", "Status pomiaru PPPM",
    "Suma opadu dzień  [mm]", "Status pomiaru WODZ",
    "Suma opadu noc   [mm]", "Status pomiaru WONO"
]

SD_COLUMNS_OF_INTEREST = [
            'Kod stacji', 'Rok', 'Miesiąc', 'Dzień',
            'Maksymalna temperatura dobowa [°C]', 'Minimalna temperatura dobowa [°C]',
            'Średnia temperatura dobowa [°C]', 'Suma dobowa opadu [mm]',
            'Usłonecznienie [godziny]', 'Aktynometria [J/cm2]',
            'Czas trwania wiatru >=10m/s [godziny]'
        ]

SDT_COLUMNS_OF_INTEREST = [
            'Kod stacji', 'Rok', 'Miesiąc', 'Dzień',
            'Średnie dobowe zachmurzenie ogólne [oktanty]',
            'Średnia dobowa prędkość wiatru [m/s]'
        ]

ELECTRICITY_COLUMNS = ["Datetime (UTC)", "Carbon Intensity gCO₂eq/kWh (direct)", "Carbon Intensity gCO₂eq/kWh (LCA)", "Low Carbon Percentage", "Renewable Percentage"]

EASY_ELECTRICITY_COLUMNS = ["Datetime", "CarbonIntensityDirect", "CarbonIntensityLCA",
                       "LowCarbonPercentage", "RenewablePercentage"]

NAN_VALUES = [8, 9, '']

### Database operations

In [19]:
def create_connection(db_name: str = DB_NAME) -> sqlite3.Connection:
    """Establish a connection to the SQLite database."""
    return sqlite3.connect(db_name)

def execute_query(conn: sqlite3.Connection, query: str, params: tuple = ()) -> None:
    """Execute a single SQL query."""
    cursor = conn.cursor()
    cursor.execute(query, params)
    conn.commit()

def record_exists(conn: sqlite3.Connection, table_name: str, key_columns: List[str], key_values: List) -> bool:
    """Check if a record already exists in the database."""
    placeholders = ' AND '.join([f"{col} = ?" for col in key_columns])
    query = f"SELECT 1 FROM {table_name} WHERE {placeholders} LIMIT 1"
    cursor = conn.cursor()
    cursor.execute(query, tuple(key_values))
    return cursor.fetchone() is not None



### Data Ingestion Functions

In [20]:
def add_sd_data(conn: sqlite3.Connection, csv_path: str) -> None:
    """Add SD data to the WeatherSD table."""
    df = pd.read_csv(csv_path, names=SD_COLUMNS, na_values=NAN_VALUES, encoding='ISO-8859-2')
    df['Date'] = df.apply(lambda row: f"{int(row['Rok']):04d}-{int(row['Miesiąc']):02d}-{int(row['Dzień']):02d}", axis=1)
    df_clean = df[SD_COLUMNS_OF_INTEREST]
    df_clean.columns = ['Station_ID', 'Year', 'Month', 'Day', 'Max_Daily_Temperature',
                        'Min_Daily_Temperature', 'Avg_Daily_Temperature',
                        'Total_Daily_Precipitation', 'Sunshine_Hours', 'Actinometry',
                        'Wind_Duration_Over_10m_s']
    df_clean['Date'] = df_clean.apply(lambda row: f"{int(row['Year']):04d}-{int(row['Month']):02d}-{int(row['Day']):02d}", axis=1)
    df_clean = df_clean[['Station_ID', 'Date', 'Max_Daily_Temperature', 'Min_Daily_Temperature',
                         'Avg_Daily_Temperature', 'Total_Daily_Precipitation',
                         'Sunshine_Hours', 'Actinometry', 'Wind_Duration_Over_10m_s']]

    query = '''INSERT INTO WeatherSD (Station_ID, Date, Max_Daily_Temperature, Min_Daily_Temperature,
                 Avg_Daily_Temperature, Total_Daily_Precipitation, Sunshine_Hours, Actinometry,
                 Wind_Duration_Over_10m_s)
               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'''

    for _, row in df_clean.iterrows():
        if not record_exists(conn, 'WeatherSD', ['Station_ID', 'Date'], [row['Station_ID'], row['Date']]):
            execute_query(conn, query, tuple(row))

def add_sdt_data(conn: sqlite3.Connection, csv_path: str) -> None:
    """Add SDT data to the WeatherSDT table."""
    df = pd.read_csv(csv_path, names=SDT_COLUMNS, na_values=NAN_VALUES, encoding='ISO-8859-2')
    df['Date'] = df.apply(lambda row: f"{int(row['Rok']):04d}-{int(row['Miesiąc']):02d}-{int(row['Dzień']):02d}", axis=1)
    df_clean = df[SDT_COLUMNS_OF_INTEREST]
    df_clean.columns = ['Station_ID', 'Year', 'Month', 'Day', 'Avg_Daily_Overall_Cloudiness',
                        'Avg_Daily_Wind_Speed']
    df_clean['Date'] = df_clean.apply(lambda row: f"{int(row['Year']):04d}-{int(row['Month']):02d}-{int(row['Day']):02d}", axis=1)
    df_clean = df_clean[['Station_ID', 'Date', 'Avg_Daily_Overall_Cloudiness', 'Avg_Daily_Wind_Speed']]

    query = '''INSERT INTO WeatherSDT (Station_ID, Date, Avg_Daily_Overall_Cloudiness, Avg_Daily_Wind_Speed)
               VALUES (?, ?, ?, ?)'''

    for _, row in df_clean.iterrows():
        if not record_exists(conn, 'WeatherSDT', ['Station_ID', 'Date'], [row['Station_ID'], row['Date']]):
            execute_query(conn, query, tuple(row))
    print(f"Added SDT data from {csv_path}")


def add_electricity_data(conn: sqlite3.Connection, csv_path: str) -> None:
    """Add Electricity data to the ElectricityData table."""
    df = pd.read_csv(csv_path, usecols=ELECTRICITY_COLUMNS, na_values=NAN_VALUES)

    query = '''INSERT INTO ElectricityData (Datetime, CarbonIntensityDirect, CarbonIntensityLCA,
                 LowCarbonPercentage, RenewablePercentage)
               VALUES (?, ?, ?, ?, ?)'''

    for _, row in df.iterrows():
        if not record_exists(conn, 'ElectricityData', ['Datetime'], [row['Datetime (UTC)']]):
            execute_query(conn, query, tuple(row))


In [21]:
# clear the tables
def clear_table(conn: sqlite3.Connection, table_name: str) -> None:
    """Clear all records from the specified table."""
    execute_query(conn, f"DELETE FROM {table_name}")


## Download and Process Data

In [22]:
DB_NAME = 'data/synop/synop.db'
BASE_URL = 'https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/synop/{year}/'
TEMP_DIR = 'data/synop/imgw/{year}/'

import os
import requests
import zipfile
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
from typing import List


In [23]:
def get_zip_urls(base_url: str) -> List[str]:
    """Scrape and return all zip file URLs from the specified base URL."""
    print(f"Fetching ZIP URLs from: {base_url}")
    response = requests.get(base_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract the URLs correctly
    urls = [base_url + a['href'] for a in soup.find_all('a') if a.get('href') and a['href'].endswith('.zip')]
    print(f"Found {len(urls)} ZIP URLs")
    return urls

def download_and_extract_zip(url: str, temp_dir: str) -> List[str]:
    """Download and extract a zip file from the given URL to the specified temporary directory."""
    print(f"Downloading and extracting ZIP from: {url}")
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    zip_name = os.path.join(temp_dir, os.path.basename(url))
    extracted_files = []

    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(zip_name, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {zip_name}")

        with zipfile.ZipFile(zip_name, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
            extracted_files = zip_ref.namelist()
        print(f"Extracted {len(extracted_files)} files from {zip_name}")

        os.remove(zip_name)
        print(f"Deleted ZIP file {zip_name}")
    except Exception as e:
        print(f"Failed to download or extract {url}: {e}")

    return [os.path.join(temp_dir, file) for file in extracted_files]

def process_extracted_files(conn: sqlite3.Connection, extracted_files: List[str]) -> None:
    """Process extracted files and add data to the respective tables."""
    for file in extracted_files:
        print(f"Processing file: {file}")
        if 's_d_' in file and not 's_d_t_' in file:
            print(f"Adding SD data from file: {file}")
            add_sd_data(conn, file)
        elif 's_d_t_' in file:
            print(f"Adding SDT data from file: {file}")
            add_sdt_data(conn, file)
        os.remove(file)
        print(f"Deleted file {file}")


def process_data_for_year(year: int, codes: List[str]) -> None:
    """Get zip files for a specified year, filter by station codes, and process data."""
    base_url = BASE_URL.format(year=year)
    temp_dir = TEMP_DIR.format(year=year)

    zip_urls = get_zip_urls(base_url)
    relevant_zip_urls = [url for url in zip_urls if any(code in url for code in codes)]
    print(f"Relevant ZIP URLs: {relevant_zip_urls}")

    conn = create_connection()
    if conn is None:
        print("Failed to create database connection")
        return

    try:
        for url in relevant_zip_urls:
            extracted_files = download_and_extract_zip(url, temp_dir)
            process_extracted_files(conn, extracted_files)
    finally:
        conn.close()
        print(f"Processed data for year {year}")

In [24]:
if __name__ == '__main__':
    # Read codes of interest from file 'data/synop/stations/station_info.csv'
    station_info = pd.read_csv('data/synop/stations/station_info.csv')
    codes_of_interest = station_info['METEO_code'].tolist()
    codes = [str(code) for code in codes_of_interest]
    print(f"Codes of interest: {codes}")
    for year in range(2021, 2024):
        process_data_for_year(year, codes)

Codes of interest: ['295', '235', '300', '500', '435', '650', '560', '520', '100', '105', '566', '690', '465', '280', '530', '330', '195', '205', '510', '399', '250', '375', '497', '625', '400']
Fetching ZIP URLs from: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/synop/2021/
Found 58 ZIP URLs
Relevant ZIP URLs: ['https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/synop/2021/2021_100_s.zip', 'https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/synop/2021/2021_105_s.zip', 'https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/synop/2021/2021_195_s.zip', 'https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/synop/2021/2021_205_s.zip', 'https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/synop/2021/2021_235_s.zip', 'https://danepubliczne.imgw.pl/data/dane

/tmp/ipykernel_711180/2355723765.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Date'] = df_clean.apply(lambda row: f"{int(row['Year']):04d}-{int(row['Month']):02d}-{int(row['Day']):02d}", axis=1)
/tmp/ipykernel_711180/2355723765.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Date'] = df_clean.apply(lambda row: f"{int(row['Year']):04d}-{int(row['Month']):02d}-{int(row['Day']):02d}", axis=1)


Deleted file data/synop/imgw/2021/s_d_100_2021.csv
Processing file: data/synop/imgw/2021/s_d_t_100_2021.csv
Adding SDT data from file: data/synop/imgw/2021/s_d_t_100_2021.csv
Added SDT data from data/synop/imgw/2021/s_d_t_100_2021.csv
Deleted file data/synop/imgw/2021/s_d_t_100_2021.csv
Downloaded data/synop/imgw/2021/2021_105_s.zip
Extracted 2 files from data/synop/imgw/2021/2021_105_s.zip
Deleted ZIP file data/synop/imgw/2021/2021_105_s.zip
Processing file: data/synop/imgw/2021/s_d_105_2021.csv
Adding SD data from file: data/synop/imgw/2021/s_d_105_2021.csv
Deleted file data/synop/imgw/2021/s_d_105_2021.csv
Processing file: data/synop/imgw/2021/s_d_t_105_2021.csv
Adding SDT data from file: data/synop/imgw/2021/s_d_t_105_2021.csv


/tmp/ipykernel_711180/2355723765.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Date'] = df_clean.apply(lambda row: f"{int(row['Year']):04d}-{int(row['Month']):02d}-{int(row['Day']):02d}", axis=1)
/tmp/ipykernel_711180/2355723765.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Date'] = df_clean.apply(lambda row: f"{int(row['Year']):04d}-{int(row['Month']):02d}-{int(row['Day']):02d}", axis=1)


Added SDT data from data/synop/imgw/2021/s_d_t_105_2021.csv
Deleted file data/synop/imgw/2021/s_d_t_105_2021.csv
Downloaded data/synop/imgw/2021/2021_195_s.zip
Extracted 2 files from data/synop/imgw/2021/2021_195_s.zip
Deleted ZIP file data/synop/imgw/2021/2021_195_s.zip
Processing file: data/synop/imgw/2021/s_d_195_2021.csv
Adding SD data from file: data/synop/imgw/2021/s_d_195_2021.csv
Deleted file data/synop/imgw/2021/s_d_195_2021.csv
Processing file: data/synop/imgw/2021/s_d_t_195_2021.csv
Adding SDT data from file: data/synop/imgw/2021/s_d_t_195_2021.csv


...

/tmp/ipykernel_711180/2355723765.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Date'] = df_clean.apply(lambda row: f"{int(row['Year']):04d}-{int(row['Month']):02d}-{int(row['Day']):02d}", axis=1)


Downloaded data/synop/imgw/2023/2023_690_s.zip
Extracted 2 files from data/synop/imgw/2023/2023_690_s.zip
Deleted ZIP file data/synop/imgw/2023/2023_690_s.zip
Processing file: data/synop/imgw/2023/s_d_690_2023.csv
Adding SD data from file: data/synop/imgw/2023/s_d_690_2023.csv


/tmp/ipykernel_711180/2355723765.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Date'] = df_clean.apply(lambda row: f"{int(row['Year']):04d}-{int(row['Month']):02d}-{int(row['Day']):02d}", axis=1)


Deleted file data/synop/imgw/2023/s_d_690_2023.csv
Processing file: data/synop/imgw/2023/s_d_t_690_2023.csv
Adding SDT data from file: data/synop/imgw/2023/s_d_t_690_2023.csv
Added SDT data from data/synop/imgw/2023/s_d_t_690_2023.csv
Deleted file data/synop/imgw/2023/s_d_t_690_2023.csv
Processed data for year 2023


/tmp/ipykernel_711180/2355723765.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Date'] = df_clean.apply(lambda row: f"{int(row['Year']):04d}-{int(row['Month']):02d}-{int(row['Day']):02d}", axis=1)


In [25]:
# add all electricity data 


if __name__ == '__main__':
    conn = create_connection()
    try:
        add_electricity_data(conn, 'data/electricity/PL_2021_daily.csv')
        add_electricity_data(conn, 'data/electricity/PL_2022_daily.csv')
        add_electricity_data(conn, 'data/electricity/PL_2023_daily.csv')
    finally:
        conn.close()

## Aggregate to csv

In [26]:
import sqlite3
import pandas as pd

DB_NAME = 'data/synop/synop.db'

def fetch_data(year: int) -> pd.DataFrame:
    """Fetch data from the WeatherSD and WeatherSDT tables for the given year."""
    conn = sqlite3.connect(DB_NAME)
    query_sd = f"""
    SELECT 
        Date, 
        MAX(Max_Daily_Temperature) AS Max_Daily_Temperature,
        MIN(Min_Daily_Temperature) AS Min_Daily_Temperature,
        AVG(Avg_Daily_Temperature) AS Avg_Daily_Temperature,
        AVG(Total_Daily_Precipitation) AS Total_Daily_Precipitation,
        AVG(Sunshine_Hours) AS Sunshine_Hours,
        AVG(Actinometry) AS Actinometry,
        AVG(Wind_Duration_Over_10m_s) AS Wind_Duration_Over_10m_s
    FROM WeatherSD
    WHERE strftime('%Y', Date) = '{year}'
    GROUP BY Date
    """

    query_sdt = f"""
    SELECT 
        Date, 
        AVG(Avg_Daily_Overall_Cloudiness) AS Avg_Daily_Overall_Cloudiness,
        AVG(Avg_Daily_Wind_Speed) AS Avg_Daily_Wind_Speed
    FROM WeatherSDT
    WHERE strftime('%Y', Date) = '{year}'
    GROUP BY Date
    """

    df_sd = pd.read_sql_query(query_sd, conn)
    df_sdt = pd.read_sql_query(query_sdt, conn)
    conn.close()

    # Merge the dataframes on the Date column
    df_merged = pd.merge(df_sd, df_sdt, on='Date', how='outer')

    return df_merged



In [27]:
# join the electricity data with the weather data on the date -> only for 2023
import pandas as pd


In [28]:
import sqlite3
import pandas as pd

DB_NAME = 'data/synop/synop.db'

def fetch_data(year: int) -> pd.DataFrame:
    """Fetch data from the WeatherSD, WeatherSDT, and ElectricityData tables for the given year."""
    conn = sqlite3.connect(DB_NAME)
    
    query_sd = f"""
    SELECT 
        Date, 
        MAX(Max_Daily_Temperature) AS Max_Daily_Temperature,
        MIN(Min_Daily_Temperature) AS Min_Daily_Temperature,
        AVG(Avg_Daily_Temperature) AS Avg_Daily_Temperature,
        AVG(Total_Daily_Precipitation) AS Total_Daily_Precipitation,
        AVG(Sunshine_Hours) AS Sunshine_Hours,
        AVG(Actinometry) AS Actinometry,
        AVG(Wind_Duration_Over_10m_s) AS Wind_Duration_Over_10m_s
    FROM WeatherSD
    WHERE strftime('%Y', Date) = '{year}'
    GROUP BY Date
    """
    
    query_sdt = f"""
    SELECT 
        Date, 
        AVG(Avg_Daily_Overall_Cloudiness) AS Avg_Daily_Overall_Cloudiness,
        AVG(Avg_Daily_Wind_Speed) AS Avg_Daily_Wind_Speed
    FROM WeatherSDT
    WHERE strftime('%Y', Date) = '{year}'
    GROUP BY Date
    """
    
    query_electricity = f"""
    SELECT 
        DATE(Datetime) AS Date, 
        AVG(CarbonIntensityDirect) AS CarbonIntensityDirect,
        AVG(CarbonIntensityLCA) AS CarbonIntensityLCA,
        AVG(LowCarbonPercentage) AS LowCarbonPercentage,
        AVG(RenewablePercentage) AS RenewablePercentage
    FROM ElectricityData
    WHERE strftime('%Y', Datetime) = '{year}'
    GROUP BY DATE(Datetime)
    """

    df_sd = pd.read_sql_query(query_sd, conn)
    df_sdt = pd.read_sql_query(query_sdt, conn)
    df_electricity = pd.read_sql_query(query_electricity, conn)
    conn.close()

    # Merge the dataframes on the Date column
    df_weather = pd.merge(df_sd, df_sdt, on='Date', how='outer')
    df_merged = pd.merge(df_weather, df_electricity, on='Date', how='outer')

    return df_merged


def save_to_csv(df: pd.DataFrame, filename: str, path='data/synop/') -> None:
    """Save the DataFrame to a CSV file."""
    df.to_csv(f"{path}{filename}", index=False)
    print(f"Data saved to {filename}")

if __name__ == '__main__':
    year = 2022
    aggregated_data = fetch_data(year)
    save_to_csv(aggregated_data, f'aggregated_data_{year}.csv')

Data saved to aggregated_data_2022.csv


In [29]:
# iterate over the years and save the data to one csv file
import os
import pandas as pd

DATA_PATH = 'data/synop/'
YEARS = [2021, 2022, 2023]

def combine_data(years: list, path: str) -> pd.DataFrame:
    """Combine data from multiple years into a single DataFrame."""
    dfs = []
    for year in years:
        filename = f'aggregated_data_{year}.csv'
        df = pd.read_csv(os
                            .path
                            .join(path, filename))
        
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

if __name__ == '__main__':
    # iterate over the years to fetch the data
    for year in YEARS:
        aggregated_data = fetch_data(year)
        save_to_csv(aggregated_data, f'aggregated_data_{year}.csv')
    aggregated_data = combine_data(YEARS, DATA_PATH)
    save_to_csv(aggregated_data, 'aggregated_data.csv')

Data saved to aggregated_data_2021.csv
Data saved to aggregated_data_2022.csv
Data saved to aggregated_data_2023.csv
Data saved to aggregated_data.csv
